<a href="https://colab.research.google.com/github/Antonfsp/deep_learling_for_coders/blob/main/dogs_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# My first model

In this notebook I develop my first deep learning model, following the learned in the lessons 1, 2 and 3 of the course "Deep learning for coders (2020)" teached by fast.ia .

The model will be a image clasiffier model capable of differentiate between 4 types of dogs: collie, golden retriever, german shepard and chihuahua.

### The library

We import download and import the fastbook library.

In [ ]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 720 kB 5.2 MB/s 
     |████████████████████████████████| 189 kB 44.1 MB/s 
     |████████████████████████████████| 46 kB 3.9 MB/s 
     |████████████████████████████████| 1.2 MB 32.1 MB/s 
     |████████████████████████████████| 56 kB 4.5 MB/s 
     |████████████████████████████████| 51 kB 285 kB/s 
Mounted at /content/gdrive


In [ ]:
from fastbook import *

## Getting the data

In this case, we will download 100 images of each type of dogs, using the DuckDuckGo image downloading API.


In [ ]:
dog_types = 'collie','golden_retriever','german_shepard','chihuahua'

path = Path('dogs')

if not path.exists():
    path.mkdir()
    for o in dog_types:
        dest = (path/o)
        dest.mkdir(exist_ok=True)
        results = search_images_ddg(f'{o} dogs', max_images=100)
        download_images(dest, urls=results)



We check that the images where created

In [ ]:
fns = get_image_files(path)
fns

(#390) [Path('dogs/collie/00000038.jpg'),Path('dogs/collie/00000036.jpg'),Path('dogs/collie/00000029.jpeg'),Path('dogs/collie/00000085.jpg'),Path('dogs/collie/00000043.jpg'),Path('dogs/collie/00000095.jpg'),Path('dogs/collie/00000097.png'),Path('dogs/collie/00000070.jpg'),Path('dogs/collie/00000077.jpg'),Path('dogs/collie/00000069.jpg')...]

We check if the images can be open

In [ ]:
failed = verify_images(fns)
failed


/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


(#5) [Path('dogs/golden_retriever/00000009.jpg'),Path('dogs/golden_retriever/00000061.jpg'),Path('dogs/german_shepard/00000088.jpg'),Path('dogs/german_shepard/00000012.jpg'),Path('dogs/german_shepard/00000031.jpg')]


We delete the images that can not be opened. verify_images returns an object of type L, which includes the method .map, which calls the passed functions for each element of the collection.
---



In [ ]:
failed.map(Path.unlink);

# Setting up the model.

Now that the data is ready, we have to create a DataLoaders.

In [ ]:
dogs = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=RandomResizedCrop(224,min_scale=0.5),
    batch_tfms=aug_transforms())

Previous command creates a "template". Now we sill have to create a actual DataLoaders.

In [ ]:
dls = dogs.dataloaders(path)

We can check some images on the Validation DataLoarder which is in the DataLoarders.

In [ ]:
dls.valid.show_batch(max_n=4, nrows=1)


Also we can check the same image from the training DataLoader with multiple iterations of the item transformations.

In [ ]:
dls.train.show_batch(max_n=4, nrows=1, unique=True)

## Training the model

In [ ]:
learn = cnn_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(4)

We can check the confusion matrix.

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

Once our model is trained, we can use it to clean the data in a easier way, using the following GUI.

In [ ]:
cleaner = ImageClassifierCleaner(learn)
cleaner

Now we run the following commands to actually delete or move the images we have just selected in the previus interface.

In [ ]:
# for idx in cleaner.delete(): cleaner.fns[idx].unlink()
# for idx,cat in cleaner.change(): shutil.move(str(cleaner.fns[idx]), path/cat)

# Exporting and importing the model

Now we can export the model to use it anywhere.

In [ ]:
path = Path()
path.ls(file_exts='.pkl')

We can also import a model.

In [ ]:
learn_inf = load_learner(path/'export.pkl')